# Step 02
# Training a U-Net model

In [1]:
%matplotlib inline

In [2]:
import sys
import os

import numpy as np
import skimage.io

import tensorflow as tf

import keras.backend
import keras.callbacks
import keras.layers
import keras.models
import keras.optimizers

import utils.model_builder
import utils.data_provider
import utils.metrics
import utils.objectives
import utils.dirtools

# Uncomment the following line if you don't have a GPU
#os.environ['CUDA_VISIBLE_DEVICES'] = ''

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
keras.__version__

'2.2.4'

# Configuration

In [4]:
from config import config_vars
print(config_vars)

{'root_directory': '/storage/data/2018_tim_tracking/unet/', 'max_training_images': 0, 'create_split_files': True, 'training_fraction': 0.5, 'validation_fraction': 0.25, 'transform_images_to_PNG': True, 'pixel_depth': 8, 'min_nucleus_size': 40, 'boundary_size': 2, 'augment_images': True, 'elastic_points': 16, 'elastic_distortion': 5, 'elastic_augmentations': 10, 'learning_rate': 1e-05, 'epochs': 15, 'steps_per_epoch': 500, 'batch_size': 10, 'val_batch_size': 10, 'rescale_labels': True, 'crop_size': 256, 'cell_min_size': 40, 'boundary_boost_factor': 1, 'object_dilation': 3, 'raw_images_dir': '/storage/data/2018_tim_tracking/unet/raw_images/', 'raw_annotations_dir': '/storage/data/2018_tim_tracking/unet/raw_annotations/', 'path_files_training': '/storage/data/2018_tim_tracking/unet/training.txt', 'path_files_validation': '/storage/data/2018_tim_tracking/unet/validation.txt', 'path_files_test': '/storage/data/2018_tim_tracking/unet/test.txt', 'normalized_images_dir': '/storage/data/2018_ti

In [5]:
experiment_name = 'neutros_v3'

config_vars = utils.dirtools.setup_experiment(config_vars, experiment_name)

data_partitions = utils.dirtools.read_data_partitions(config_vars)

config_vars

{'augment_images': True,
 'batch_size': 10,
 'boundary_boost_factor': 1,
 'boundary_labels_dir': '/storage/data/2018_tim_tracking/unet/boundary_labels/',
 'boundary_size': 2,
 'cell_min_size': 40,
 'create_split_files': True,
 'crop_size': 256,
 'csv_log_file': '/storage/data/2018_tim_tracking/unet/experiments/neutros_v3/log.csv',
 'elastic_augmentations': 10,
 'elastic_distortion': 5,
 'elastic_points': 16,
 'epochs': 15,
 'experiment_dir': '/storage/data/2018_tim_tracking/unet/experiments/neutros_v3/out/',
 'labels_out_dir': '/storage/data/2018_tim_tracking/unet/experiments/neutros_v3/out/segm/',
 'learning_rate': 1e-05,
 'max_training_images': 0,
 'min_nucleus_size': 40,
 'model_file': '/storage/data/2018_tim_tracking/unet/experiments/neutros_v3/model.hdf5',
 'normalized_images_dir': '/storage/data/2018_tim_tracking/unet/norm_images/',
 'object_dilation': 3,
 'path_files_test': '/storage/data/2018_tim_tracking/unet/test.txt',
 'path_files_training': '/storage/data/2018_tim_tracking/

# Initiate data generators

In [6]:
# build session running on GPU 1
configuration = tf.ConfigProto()
configuration.gpu_options.allow_growth = True
configuration.gpu_options.visible_device_list = "0"
session = tf.Session(config = configuration)

# apply session
keras.backend.set_session(session)

train_gen = utils.data_provider.random_sample_generator(
    config_vars["normalized_images_dir"],
    config_vars["boundary_labels_dir"],
    data_partitions["training"],
    config_vars["batch_size"],
    config_vars["pixel_depth"],
    config_vars["crop_size"],
    config_vars["crop_size"],
    config_vars["rescale_labels"]
)

val_gen = utils.data_provider.single_data_from_images(
     config_vars["normalized_images_dir"],
     config_vars["boundary_labels_dir"],
     data_partitions["validation"],
     config_vars["val_batch_size"],
     config_vars["pixel_depth"],
     config_vars["crop_size"],
     config_vars["crop_size"],
     config_vars["rescale_labels"]
)

# Build model

In [ ]:
# build model
model = utils.model_builder.get_model_3_class(config_vars["crop_size"], config_vars["crop_size"], activation=None)
model.summary()

#loss = "categorical_crossentropy"
loss = utils.objectives.weighted_crossentropy

metrics = [keras.metrics.categorical_accuracy, 
           utils.metrics.channel_recall(channel=0, name="background_recall"), 
           utils.metrics.channel_precision(channel=0, name="background_precision"),
           utils.metrics.channel_recall(channel=1, name="interior_recall"), 
           utils.metrics.channel_precision(channel=1, name="interior_precision"),
           utils.metrics.channel_recall(channel=2, name="boundary_recall"), 
           utils.metrics.channel_precision(channel=2, name="boundary_precision"),
          ]

optimizer = keras.optimizers.RMSprop(lr=config_vars["learning_rate"])

model.compile(loss=loss, metrics=metrics, optimizer=optimizer)

# Performance logging
callback_csv = keras.callbacks.CSVLogger(filename=config_vars["csv_log_file"])

callbacks=[callback_csv]


/home/cells2numbers/2018_unet_segmentation_hudy/unet4nuclei/unet4nuclei/utils/model_builder.py:16: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", padding="same")`
  a = keras.layers.Convolution2D(64, 3, 3, **option_dict_conv)(x)
/home/cells2numbers/2018_unet_segmentation_hudy/unet4nuclei/unet4nuclei/utils/model_builder.py:17: UserWarning: Update your `BatchNormalization` call to the Keras 2 API: `BatchNormalization(momentum=0.9)`
  a = keras.layers.BatchNormalization(**option_dict_bn)(a)
/home/cells2numbers/2018_unet_segmentation_hudy/unet4nuclei/unet4nuclei/utils/model_builder.py:19: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", padding="same")`
  a = keras.layers.Convolution2D(64, 3, 3, **option_dict_conv)(a)
/home/cells2numbers/2018_unet_segmentation_hudy/unet4nuclei/unet4nuclei/utils/model_builder.py:20: UserWarning: Update your `BatchNormalization` call to the Keras 2 API: `Batch

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 1)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 256, 64) 640         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 256, 256, 64) 256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 256, 256, 64) 36928       batch_normalization_1[0][0]      
__________________________________________________________________________________________________
batch_norm

# Training 

In [ ]:
config_vars["epochs"] = 600
config_vars["steps_per_epoch"] = 100

# TRAIN
statistics = model.fit_generator(
    generator=train_gen,
    steps_per_epoch=config_vars["steps_per_epoch"],
    epochs=config_vars["epochs"],
    validation_data=val_gen,
    validation_steps=20, #int(len(data_partitions["validation"])/config_vars["val_batch_size"]),
    callbacks=callbacks
)

model.save_weights(config_vars["model_file"])

print('Done! :)')

Epoch 1/600
Training with 22 images.
100/100 [==============================] - 39s 389ms/step - loss: 0.7680 - categorical_accuracy: 0.8243 - background_recall: 0.8478 - background_precision: 0.9875 - interior_recall: 0.0987 - interior_precision: 0.1216 - boundary_recall: 0.7815 - boundary_precision: 0.1462 - val_loss: 0.1564 - val_categorical_accuracy: 0.9798 - val_background_recall: 0.9885 - val_background_precision: 0.9984 - val_interior_recall: 0.2993 - val_interior_precision: 0.9438 - val_boundary_recall: 0.9067 - val_boundary_precision: 0.2639
Epoch 2/600
100/100 [==============================] - 33s 332ms/step - loss: 0.5023 - categorical_accuracy: 0.9405 - background_recall: 0.9592 - background_precision: 0.9970 - interior_recall: 0.3607 - interior_precision: 0.6546 - boundary_recall: 0.9005 - boundary_precision: 0.2485 - val_loss: 0.1423 - val_categorical_accuracy: 0.9820 - val_background_recall: 0.9881 - val_background_precision: 0.9999 - val_interior_recall: 0.4865 - val_i

100/100 [==============================] - 33s 329ms/step - loss: 0.2604 - categorical_accuracy: 0.9619 - background_recall: 0.9742 - background_precision: 0.9993 - interior_recall: 0.5643 - interior_precision: 0.9475 - boundary_recall: 0.9551 - boundary_precision: 0.3248 - val_loss: 0.1149 - val_categorical_accuracy: 0.9816 - val_background_recall: 0.9870 - val_background_precision: 0.9999 - val_interior_recall: 0.5552 - val_interior_precision: 0.9181 - val_boundary_recall: 0.9387 - val_boundary_precision: 0.2787
Epoch 17/600
100/100 [==============================] - 33s 330ms/step - loss: 0.2520 - categorical_accuracy: 0.9631 - background_recall: 0.9750 - background_precision: 0.9994 - interior_recall: 0.5757 - interior_precision: 0.9517 - boundary_recall: 0.9585 - boundary_precision: 0.3328 - val_loss: 0.1127 - val_categorical_accuracy: 0.9834 - val_background_recall: 0.9894 - val_background_precision: 0.9998 - val_interior_recall: 0.5187 - val_interior_precision: 0.9213 - val_boun

100/100 [==============================] - 33s 329ms/step - loss: 0.2017 - categorical_accuracy: 0.9649 - background_recall: 0.9763 - background_precision: 0.9993 - interior_recall: 0.6001 - interior_precision: 0.9399 - boundary_recall: 0.9591 - boundary_precision: 0.3447 - val_loss: 0.1051 - val_categorical_accuracy: 0.9834 - val_background_recall: 0.9889 - val_background_precision: 0.9998 - val_interior_recall: 0.5517 - val_interior_precision: 0.9499 - val_boundary_recall: 0.9338 - val_boundary_precision: 0.2983
Epoch 32/600
100/100 [==============================] - 33s 330ms/step - loss: 0.1965 - categorical_accuracy: 0.9660 - background_recall: 0.9771 - background_precision: 0.9995 - interior_recall: 0.6077 - interior_precision: 0.9527 - boundary_recall: 0.9615 - boundary_precision: 0.3531 - val_loss: 0.1064 - val_categorical_accuracy: 0.9834 - val_background_recall: 0.9889 - val_background_precision: 0.9999 - val_interior_recall: 0.5417 - val_interior_precision: 0.9526 - val_boun

100/100 [==============================] - 33s 332ms/step - loss: 0.1626 - categorical_accuracy: 0.9682 - background_recall: 0.9789 - background_precision: 0.9995 - interior_recall: 0.6164 - interior_precision: 0.9606 - boundary_recall: 0.9639 - boundary_precision: 0.3619 - val_loss: 0.0990 - val_categorical_accuracy: 0.9845 - val_background_recall: 0.9897 - val_background_precision: 0.9998 - val_interior_recall: 0.5869 - val_interior_precision: 0.9428 - val_boundary_recall: 0.9220 - val_boundary_precision: 0.3117
Epoch 47/600
100/100 [==============================] - 33s 329ms/step - loss: 0.1631 - categorical_accuracy: 0.9669 - background_recall: 0.9780 - background_precision: 0.9995 - interior_recall: 0.6173 - interior_precision: 0.9576 - boundary_recall: 0.9627 - boundary_precision: 0.3622 - val_loss: 0.0925 - val_categorical_accuracy: 0.9855 - val_background_recall: 0.9915 - val_background_precision: 0.9997 - val_interior_recall: 0.5187 - val_interior_precision: 0.9598 - val_boun

100/100 [==============================] - 33s 331ms/step - loss: 0.1412 - categorical_accuracy: 0.9693 - background_recall: 0.9799 - background_precision: 0.9995 - interior_recall: 0.6337 - interior_precision: 0.9762 - boundary_recall: 0.9646 - boundary_precision: 0.3770 - val_loss: 0.0876 - val_categorical_accuracy: 0.9844 - val_background_recall: 0.9897 - val_background_precision: 0.9998 - val_interior_recall: 0.5795 - val_interior_precision: 0.9448 - val_boundary_recall: 0.9206 - val_boundary_precision: 0.3099
Epoch 62/600
100/100 [==============================] - 33s 330ms/step - loss: 0.1368 - categorical_accuracy: 0.9700 - background_recall: 0.9804 - background_precision: 0.9996 - interior_recall: 0.6372 - interior_precision: 0.9760 - boundary_recall: 0.9676 - boundary_precision: 0.3837 - val_loss: 0.0841 - val_categorical_accuracy: 0.9863 - val_background_recall: 0.9926 - val_background_precision: 0.9996 - val_interior_recall: 0.5004 - val_interior_precision: 0.9673 - val_boun

...

100/100 [==============================] - 33s 332ms/step - loss: 0.0797 - categorical_accuracy: 0.9801 - background_recall: 0.9887 - background_precision: 0.9995 - interior_recall: 0.7073 - interior_precision: 0.9867 - boundary_recall: 0.9630 - boundary_precision: 0.4793 - val_loss: 0.0557 - val_categorical_accuracy: 0.9886 - val_background_recall: 0.9939 - val_background_precision: 0.9994 - val_interior_recall: 0.6069 - val_interior_precision: 0.9575 - val_boundary_recall: 0.8886 - val_boundary_precision: 0.3822
Epoch 152/600
100/100 [==============================] - 33s 331ms/step - loss: 0.0805 - categorical_accuracy: 0.9799 - background_recall: 0.9886 - background_precision: 0.9995 - interior_recall: 0.7080 - interior_precision: 0.9866 - boundary_recall: 0.9621 - boundary_precision: 0.4794 - val_loss: 0.0636 - val_categorical_accuracy: 0.9891 - val_background_recall: 0.9958 - val_background_precision: 0.9988 - val_interior_recall: 0.5182 - val_interior_precision: 0.9827 - val_bou

100/100 [==============================] - 33s 331ms/step - loss: 0.0778 - categorical_accuracy: 0.9805 - background_recall: 0.9891 - background_precision: 0.9995 - interior_recall: 0.7179 - interior_precision: 0.9876 - boundary_recall: 0.9624 - boundary_precision: 0.4939 - val_loss: 0.0585 - val_categorical_accuracy: 0.9896 - val_background_recall: 0.9951 - val_background_precision: 0.9992 - val_interior_recall: 0.6060 - val_interior_precision: 0.9647 - val_boundary_recall: 0.8649 - val_boundary_precision: 0.4044
Epoch 167/600
100/100 [==============================] - 33s 329ms/step - loss: 0.0751 - categorical_accuracy: 0.9812 - background_recall: 0.9895 - background_precision: 0.9995 - interior_recall: 0.7167 - interior_precision: 0.9876 - boundary_recall: 0.9625 - boundary_precision: 0.4913 - val_loss: 0.0561 - val_categorical_accuracy: 0.9891 - val_background_recall: 0.9947 - val_background_precision: 0.9992 - val_interior_recall: 0.5943 - val_interior_precision: 0.9607 - val_bou

100/100 [==============================] - 33s 332ms/step - loss: 0.0729 - categorical_accuracy: 0.9816 - background_recall: 0.9898 - background_precision: 0.9995 - interior_recall: 0.7273 - interior_precision: 0.9879 - boundary_recall: 0.9632 - boundary_precision: 0.5040 - val_loss: 0.0609 - val_categorical_accuracy: 0.9892 - val_background_recall: 0.9947 - val_background_precision: 0.9990 - val_interior_recall: 0.6164 - val_interior_precision: 0.9517 - val_boundary_recall: 0.8357 - val_boundary_precision: 0.3913
Epoch 182/600
100/100 [==============================] - 33s 333ms/step - loss: 0.0746 - categorical_accuracy: 0.9811 - background_recall: 0.9895 - background_precision: 0.9994 - interior_recall: 0.7264 - interior_precision: 0.9883 - boundary_recall: 0.9622 - boundary_precision: 0.5020 - val_loss: 0.0585 - val_categorical_accuracy: 0.9896 - val_background_recall: 0.9954 - val_background_precision: 0.9989 - val_interior_recall: 0.5912 - val_interior_precision: 0.9680 - val_bou

100/100 [==============================] - 33s 331ms/step - loss: 0.0695 - categorical_accuracy: 0.9823 - background_recall: 0.9902 - background_precision: 0.9995 - interior_recall: 0.7334 - interior_precision: 0.9881 - boundary_recall: 0.9639 - boundary_precision: 0.5101 - val_loss: 0.0545 - val_categorical_accuracy: 0.9895 - val_background_recall: 0.9948 - val_background_precision: 0.9992 - val_interior_recall: 0.6316 - val_interior_precision: 0.9534 - val_boundary_recall: 0.8538 - val_boundary_precision: 0.4003
Epoch 197/600
100/100 [==============================] - 33s 331ms/step - loss: 0.0722 - categorical_accuracy: 0.9815 - background_recall: 0.9898 - background_precision: 0.9995 - interior_recall: 0.7361 - interior_precision: 0.9882 - boundary_recall: 0.9633 - boundary_precision: 0.5134 - val_loss: 0.0597 - val_categorical_accuracy: 0.9887 - val_background_recall: 0.9941 - val_background_precision: 0.9992 - val_interior_recall: 0.6312 - val_interior_precision: 0.9338 - val_bou

100/100 [==============================] - 33s 329ms/step - loss: 0.0687 - categorical_accuracy: 0.9823 - background_recall: 0.9902 - background_precision: 0.9995 - interior_recall: 0.7436 - interior_precision: 0.9883 - boundary_recall: 0.9651 - boundary_precision: 0.5231 - val_loss: 0.0584 - val_categorical_accuracy: 0.9896 - val_background_recall: 0.9952 - val_background_precision: 0.9989 - val_interior_recall: 0.6303 - val_interior_precision: 0.9379 - val_boundary_recall: 0.8085 - val_boundary_precision: 0.3986
Epoch 212/600
100/100 [==============================] - 33s 331ms/step - loss: 0.0689 - categorical_accuracy: 0.9822 - background_recall: 0.9902 - background_precision: 0.9995 - interior_recall: 0.7451 - interior_precision: 0.9891 - boundary_recall: 0.9657 - boundary_precision: 0.5246 - val_loss: 0.0582 - val_categorical_accuracy: 0.9901 - val_background_recall: 0.9960 - val_background_precision: 0.9986 - val_interior_recall: 0.5973 - val_interior_precision: 0.9724 - val_bou